In [ ]:
import numpy as np
import os
import pandas as pd
import re
import requests
import shutil
import zipfile
import glob

from tqdm import tqdm

# Présidentielles et législatives
D'après les données collationnées par Piketty et Cagé pour leur ouvrage *[Une histoire du conflit politique](https://unehistoireduconflitpolitique.fr/)*.
## 1. Téléchargement et préparation

In [ ]:
def extract_strings_from_webpage(url):
    response = requests.get(url) 
    if response.status_code == 200:
        strings = re.findall(r'"([^"]*)"', response.text)
        return strings
    else:
        print(f'Failed to fetch the webpage. Status code: {response.status_code}')
        return []

webpage_url = 'https://unehistoireduconflitpolitique.fr/telecharger.html'
extracted_strings = extract_strings_from_webpage(webpage_url)
download_links = [item for item in extracted_strings if item.endswith('dta.zip') or item.endswith('dta.zip')]
print(f'Identified {len(download_links)} files to download.')

In [ ]:
filtered_links = [
    link for link in download_links 
    if ('pres' in link.lower() or 'leg' in link.lower()) 
    and any(int(match.group(1)) >= 1981 
           for match in re.finditer(r'(?:pres|leg)(\d{4})', link.lower()))
]
print(f'Extracted {len(filtered_links)} relevant files.')

In [ ]:
os.makedirs('data/elec/zip', exist_ok=True)

progress_bar = tqdm(total=len(filtered_links), desc='Downloading', unit='file')

for link in filtered_links:
    try:
        file_name = os.path.join('data/elec/zip', os.path.basename(link))
        response = requests.get(link)
        with open(file_name, 'wb') as file:
            file.write(response.content)
        progress_bar.update(1)
    except Exception as e:
        print(f'Error downloading {link}: {e}')
        
progress_bar.close()

In [ ]:
zip_dir = 'data/elec/zip'
dta_dir = 'data/elec/dta'

os.makedirs(dta_dir, exist_ok=True)

zip_files = [file for file in os.listdir(zip_dir) if file.endswith('.zip')]
progress_bar = tqdm(total=len(zip_files), desc="Extracting", unit="file")

for file in zip_files:
    try:
        zip_file_path = os.path.join(zip_dir, file)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.infolist():
                basename = os.path.basename(member.filename)
                if basename.startswith('._'):
                    continue
                
                if member.filename.lower().endswith(('.dta', '.csv')):
                    target_path = os.path.join(dta_dir, basename)
                    with zip_ref.open(member) as source, open(target_path, 'wb') as dest:
                        shutil.copyfileobj(source, dest)

        progress_bar.update(1)
    except Exception as e:
        print(f'Error extracting {file}: {e}')
        
progress_bar.close()

In [ ]:
for prefix in ["leg", "pres"]:
    files = glob.glob(f"data/elec/dta/{prefix}????comm.dta") + glob.glob(f"data/elec/dta/{prefix}????comm.csv")

    # Step 1: collect all voix columns across all files
    all_candidates = set()
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext == '.dta':
            df = pd.read_stata(file)
        elif ext == '.csv':
            df = pd.read_csv(file)
        candidates = [col for col in df.columns if col.startswith('voix')]
        all_candidates.update(candidates)
    all_candidates = list(all_candidates)
    
    # Step 2: compute sums and proportions
    counts_records = []
    shares_records = []
    
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext == '.dta':
            df = pd.read_stata(file)
        elif ext == '.csv':
            df = pd.read_csv(file)
        match = re.search(rf'{prefix}(\d{{4}})comm\.(dta|csv)', os.path.basename(file))
        year = int(match.group(1)) if match else None

        for col in all_candidates:
            if col not in df.columns:
                df[col] = np.nan

        sums = df[all_candidates + ['inscrits']].sum(min_count=1)
        tot_inscrits = sums['inscrits']

        for col in all_candidates:
            candidate_name = col.replace('voix', '')
            counts_records.append({
                'candidat': candidate_name,
                str(year): sums[col]
            })
            shares_records.append({
                'candidat': candidate_name,
                str(year): sums[col] / tot_inscrits if tot_inscrits != 0 else 0
            })

    # Step 3: build dataframes for counts and shares
    df_counts_raw = pd.DataFrame(counts_records)
    df_counts = df_counts_raw.groupby('candidat').first().reset_index()

    df_shares_raw = pd.DataFrame(shares_records)
    df_shares = df_shares_raw.groupby('candidat').first().reset_index()

    # Reorder columns
    year_columns = sorted([col for col in df_counts.columns if col != 'candidat'])
    df_counts = df_counts[['candidat'] + year_columns]
    df_shares = df_shares[['candidat'] + year_columns]

    # Step 4: handle round info for presidential elections
    if prefix == "pres":
        for df in [df_counts, df_shares]:
            df['tour'] = np.where(df['candidat'].str.contains('T2'), 2, 1)
            df['candidat'] = df['candidat'].str.replace('T2', '', regex=False)

    # Step 5: save to parquet
    df_counts.to_parquet(f"data/elec/{prefix}_counts.parquet", index=False)
    df_shares.to_parquet(f"data/elec/{prefix}_shares.parquet", index=False)

In [ ]:
shutil.rmtree("data/elec/zip")
shutil.rmtree("data/elec/dta")

## 2. Nuances politiques pour les élections législatives
D'après l'encodage documenté dans les [annexes](https://conflit-politique-data.ams3.cdn.digitaloceanspaces.com/pdf/CagePiketty2023Annexes.pdf).
- AUG = autre gauche non-communiste
- AUD = partis et organisations à la droite de la droite
- CENALLI = CEN-ALLI (Le Centre pour la France / Alliance Centriste)
- CPNT = Chasse, pêche, nature et traditions
- DIV = candidats divers et inclassables (régionalistes, animalistes...)
- DLF = Debout la France
- DVD = divers droite
- DVG = divers gauche
- ECO = candidats écologistes (hors NUPES)
- EELV = Europe Ecologie Les Verts
- ENS = Ensemble (Renaissance/LRM, Modem, Horizons)
- FG = Front de Gauche (PCF, PG, DVG)
- FN = Front National
- GEC = Génération Ecologie + divers écologiques hormis les Verts
- LCR = Ligue Communiste Révolutionnaire
- LFI = La France Insoumise
- LO = Lutte Ouvrière
- LR = Les Républicains
- LREM = La République en Marche
- MAJ = majorité présidentielle hors PS / RDG
- MDM = Mouvement Démocrate / MoDem
- MNRAUD = MNR-AUD = Mouvement National Républicain et autres organisations de droite
- MPF = Mouvement pour la France
- NCE = Nouveau Centre
- NUP = NUPES (LFI, PS, EELV, PCF)
- PCF = Parti Communiste Français
- PREP = Pôle Républicain (MDC et autres partis de la mouvance républicaine)
- PRGDVG = PRG-DVG = Parti Radical de Gauche et divers gauche
- PRV = Parti Radical
- PS = Parti Socialiste
- RDG = Radicaux de Gauche
- RDGDVG = RDG-DVG = Radicaux de gauche et divers gauche
- REC = Reconquête
- REG = régionalistes
- RN = Rassemblement National
- RPR = Rassemblement pour la République
- UDF = Union pour la Démocratie Française
- UDFD = Union pour la Démocratie Française et MoDem
- UDFRPR = UDF-RPR
- UDI = Union des Démocrates et Indépendants
- UMP = Union pour un Mouvement Populaire
- VEC = Les Verts et organisations proches

In [ ]:
leg_counts = pd.read_parquet("data/elec/leg_counts.parquet")
leg_shares = pd.read_parquet("data/elec/leg_shares.parquet")

In [ ]:
alignment_dict = {
    # autre
    'DIV': 'autre',
    'REG': 'autre',
    'ECO': 'autre',
    # centre
    'CENALLI': 'centre',
    'ENS': 'centre',
    'LREM': 'centre',
    'PREP': 'centre',
    # centredroite_droite
    'CPNT': 'centredroite_droite',
    'DVD': 'centredroite_droite',
    'MDM': 'centredroite_droite',
    'NCE': 'centredroite_droite',
    'LR': 'centredroite_droite',
    'PRV': 'centredroite_droite',
    'RPR': 'centredroite_droite',
    'UDF': 'centredroite_droite',
    'UDFD': 'centredroite_droite',
    'UDFRPR': 'centredroite_droite',
    'UDI': 'centredroite_droite',
    'UMP': 'centredroite_droite',
    # centregauche_gauche
    'DVG': 'centregauche_gauche',
    'EELV': 'centregauche_gauche',
    'FG': 'centregauche_gauche',
    'GEC': 'centregauche_gauche',
    'MAJ': 'centregauche_gauche',
    'NUP': 'centregauche_gauche',
    'PRGDVG': 'centregauche_gauche',
    'PS': 'centregauche_gauche',
    'RDGDVG': 'centregauche_gauche',
    'RDG': 'centregauche_gauche',
    'VEC': 'centregauche_gauche',
    # extremedroite_droiteradicale
    'AUD': 'extremedroite_droiteradicale',
    'DLF': 'extremedroite_droiteradicale',
    'FN': 'extremedroite_droiteradicale',
    'MNRAUD': 'extremedroite_droiteradicale',
    'MPF': 'extremedroite_droiteradicale',
    'REC': 'extremedroite_droiteradicale',
    'RN': 'extremedroite_droiteradicale',
    # extremegauche_gaucheradicale
    'AUG': 'extremegauche_gaucheradicale',
    'LCR': 'extremegauche_gaucheradicale',
    'LFI': 'extremegauche_gaucheradicale',
    'LO': 'extremegauche_gaucheradicale',
    'PCF': 'extremegauche_gaucheradicale'
}

leg_counts['political_alignment'] = leg_counts['candidat'].map(alignment_dict)
leg_shares['political_alignment'] = leg_shares['candidat'].map(alignment_dict)

In [ ]:
leg_counts.to_parquet(f"data/elec/leg_counts.parquet", index=False)
leg_shares.to_parquet(f"data/elec/leg_shares.parquet", index=False)

## 3. Nuances politiques pour les élections présidentielles

In [ ]:
pres_counts = pd.read_parquet("data/elec/pres_counts.parquet")
pres_shares = pd.read_parquet("data/elec/pres_shares.parquet")

In [ ]:
alignment_dict = {
    # autre
    'LALONDE': 'autre',
    'LEPAGE': 'autre',
    'NIHOUS': 'autre',
    'SAINTJOSSE': 'autre',
    
    # centre
    'MACRON': 'centre',
    
    # centredroite_droite
    'BALLADUR': 'centredroite_droite',
    'BAYROU': 'centredroite_droite',
    'CHIRAC': 'centredroite_droite',
    'DEBRE': 'centredroite_droite',
    'FILLON': 'centredroite_droite',
    'GARAUD': 'centredroite_droite',
    'GISCARDDESTAING': 'centredroite_droite',
    'LASSALLE': 'centredroite_droite',
    'MADELIN': 'centredroite_droite',
    'PECRESSE': 'centredroite_droite',
    'SARKOZY': 'centredroite_droite',
    
    # centregauche_gauche
    'CHEVENEMENT': 'centregauche_gauche',
    'CREPEAU': 'centregauche_gauche',
    'HAMON': 'centregauche_gauche',
    'HIDALGO': 'centregauche_gauche',
    'HOLLANDE': 'centregauche_gauche',
    'JADOT': 'centregauche_gauche',
    'JOLY': 'centregauche_gauche',
    'JOSPIN': 'centregauche_gauche',
    'MAMERE': 'centregauche_gauche',
    'MITTERRAND': 'centregauche_gauche',
    'ROYAL': 'centregauche_gauche',
    'TAUBIRA': 'centregauche_gauche',
    'VOYNET': 'centregauche_gauche',
    
    # extremedroite_droiteradicale
    'ASSELINEAU': 'extremedroite_droiteradicale',
    'BOUTIN': 'extremedroite_droiteradicale',
    'CHEMINADE': 'extremedroite_droiteradicale',
    'DUPONTAIGNAN': 'extremedroite_droiteradicale',
    'LEPEN': 'extremedroite_droiteradicale',
    'MEGRET': 'extremedroite_droiteradicale',
    'MLEPEN': 'extremedroite_droiteradicale',
    'VILLIERS': 'extremedroite_droiteradicale',
    'ZEMMOUR': 'extremedroite_droiteradicale',
    
    # extremegauche_gaucheradicale
    'ARTHAUD': 'extremegauche_gaucheradicale',
    'BESANCENOT': 'extremegauche_gaucheradicale',
    'BOUCHARDEAU': 'extremegauche_gaucheradicale',
    'BOVE': 'extremegauche_gaucheradicale',
    'BUFFET': 'extremegauche_gaucheradicale',
    'GLUCKSTEIN': 'extremegauche_gaucheradicale',
    'HUE': 'extremegauche_gaucheradicale',
    'LAGUILLER': 'extremegauche_gaucheradicale',
    'MARCHAIS': 'extremegauche_gaucheradicale',
    'MELENCHON': 'extremegauche_gaucheradicale',
    'POUTOU': 'extremegauche_gaucheradicale',
    'ROUSSEL': 'extremegauche_gaucheradicale',
    'SCHIVARDI': 'extremegauche_gaucheradicale',
}

pres_counts['political_alignment'] = pres_counts['candidat'].map(alignment_dict)
pres_shares['political_alignment'] = pres_shares['candidat'].map(alignment_dict)

In [ ]:
pres_counts.to_parquet(f"data/elec/pres_counts.parquet", index=False)
pres_shares.to_parquet(f"data/elec/pres_shares.parquet", index=False)

# Européennes
Résultats téléchargés depuis [data.gouv.fr](https://www.data.gouv.fr/).

In [ ]:
urls = {
    2019: "https://static.data.gouv.fr/resources/resultats-des-elections-europeennes-2019/20190531-144431/resultats-definitifs-par-region.xls",
    2014: "https://www.data.gouv.fr/storage/f/2014-05-30T10-34-25/euro-2014-resultats-c.xlsx",
    2009: "https://static.data.gouv.fr/e0/88c770f067e9bc4cfd3dea656aa69b15add243c1c4c367a85d7314e74fde63.xls",
    2004: "https://static.data.gouv.fr/82/b8b9cde91b95802dc0092d4a76c10dbf5b0b0fae8e4ccafc6ffb33397e8053.xls",
    1999: "https://static.data.gouv.fr/fa/d907ec8071c5f153de8235efa65df560bb269425132521c079657794a0c62a.xls"
}

os.makedirs('data/xls', exist_ok=True)

for year, url in urls.items():
    response = requests.get(url)
    response.raise_for_status()
    filepath = f"data/xls/{year}.xls"
    with open(filepath, "wb") as f:
        f.write(response.content)
    print(f"Downloaded {year}: {filepath}")

In [ ]:
df = pd.read_excel("data/xls/2019.xls")

In [ ]:
df = df.drop(columns=[
    "Code de la région", "Abstentions", "% Abs/Ins", 
    "Votants", "% Vot/Ins", "Blancs", "% Blancs/Ins", "% Blancs/Vot", 
    "Nuls", "% Nuls/Ins", "% Nuls/Vot", "Exprimés", "% Exp/Ins", "% Exp/Vot"
])

col_names = ["N°Liste", "Libellé Abrégé Liste", "Liste",
             "Nom Tête de Liste", "Voix", "% Voix/Ins", "% Voix/Exp"]
data_columns = df.columns[2:]
n_lists = len(data_columns) // len(col_names)

renamed_columns = []
for i in range(n_lists):
    for name in col_names:
        renamed_columns.append(f"{name}")

df.columns = ['Région'] + ['Inscrits'] + renamed_columns

In [ ]:
fixed_cols = df.iloc[:, :2]

blocks = []
for i in range(34):
    start = 2 + i * 7
    end = start + 7
    block = df.iloc[:, start:end].copy()
    block.columns = ["N°Liste", "Libellé Abrégé Liste", "Liste",  
                     "Nom Tête de Liste", "Voix", "% Voix/Ins", "% Voix/Exp"]
    block["Région"] = fixed_cols["Région"]
    block["Inscrits"] = fixed_cols["Inscrits"]
    blocks.append(block)

long_df = pd.concat(blocks, ignore_index=True)

In [ ]:
summary = (
    long_df.groupby(["Liste"])
    .agg({
        "Inscrits": "sum",
        "Voix": "sum"
    })
    .reset_index()
)

summary["Voix/Inscrits"] = summary["Voix"] / summary["Inscrits"]
summary = summary.drop(columns = 'Inscrits')

In [ ]:
shutil.rmtree("data/xls")